In [21]:
import pandas as pd

# 파일 읽기 (인코딩 지정)
df = pd.read_csv('마늘.csv', encoding='cp949')

In [ ]:

# 데이터프레임을 df라고 가정
# 남길 품목명들을 리스트로 정의
keep_items = ['깐마늘', '깐마늘 남도', '깐마늘 대서']

# isin() 메서드로 해당 품목들만 필터링
df_filtered = df[df['품목명'].isin(keep_items)]

In [23]:
import pandas as pd

# pandas 출력 옵션 설정 (모든 행과 열 표시)
pd.set_option('display.max_rows', None)      # 모든 행 출력
pd.set_option('display.max_columns', None)   # 모든 열 출력
pd.set_option('display.width', None)         # 너비 제한 해제
pd.set_option('display.max_colwidth', None)  # 컬럼 너비 제한 해제

# 이후 출력
print(df_filtered)
df_filtered.to_csv('깐마늘머지.csv', index=False, encoding='utf-8-sig')

             거래일자     품목명      반입량           금액   전년 반입량  전년 반입량 증감률(%)  \
3654   2025-06-30     깐마늘      NaN          NaN      NaN            0.0   
3655   2025-06-29     깐마늘      NaN          NaN      NaN            0.0   
3656   2025-06-28     깐마늘      NaN          NaN      NaN            0.0   
3657   2025-06-27     깐마늘      NaN          NaN      NaN            0.0   
3658   2025-06-26     깐마늘      NaN          NaN      NaN            0.0   
3659   2025-06-25     깐마늘      NaN          NaN      NaN            0.0   
3660   2025-06-24     깐마늘      NaN          NaN      NaN            0.0   
3661   2025-06-23     깐마늘      NaN          NaN      NaN            0.0   
3662   2025-06-22     깐마늘      NaN          NaN      NaN            0.0   
3663   2025-06-21     깐마늘      NaN          NaN      NaN            0.0   
3664   2025-06-20     깐마늘      NaN          NaN      NaN            0.0   
3665   2025-06-19     깐마늘      NaN          NaN      NaN            0.0   
3666   2025-06-18     깐마늘

마늘 데이터 세부품목 정리 후 필요 품목 남긴 파일 깐마늘머지.csv로 저장

In [28]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

def integrate_garlic_data_extended(df):
    """
    깐마늘 관련 데이터를 통합하고 전년대비, 평년대비 증감률까지 모두 계산하는 함수
    """
    print("=== 확장된 깐마늘 데이터 통합 시작 ===")
    
    # 1️⃣ 기본 데이터 전처리
    df['거래일자'] = pd.to_datetime(df['거래일자'])
    target_items = ['깐마늘', '깐마늘 대서', '깐마늘 남도']
    filtered_df = df[df['품목명'].isin(target_items)].copy()
    
    print(f"대상 품목 데이터: {len(filtered_df)}개 행")
    
    # 2️⃣ 전체 컬럼 선택 (기존 + 추가 컬럼들)
    all_cols = ['거래일자', '품목명', '반입량', '전년 반입량', '전년 반입량 증감률(%)', 
                '금액', '전년 금액', '전년 금액 증감률(%)', 
                '평년 반입량', '평년 반입량 증감률(%)', 
                '평년 금액', '평년 금액 증감률(%)']
    
    # 사용 가능한 컬럼만 선택
    available_cols = [col for col in all_cols if col in filtered_df.columns]
    work_df = filtered_df[available_cols].copy()
    
    print(f"사용 가능한 컬럼: {available_cols}")
    
    # 3️⃣ 숫자형 컬럼들을 모두 숫자로 변환
    numeric_cols = ['반입량', '전년 반입량', '전년 반입량 증감률(%)', 
                    '금액', '전년 금액', '전년 금액 증감률(%)', 
                    '평년 반입량', '평년 반입량 증감률(%)', 
                    '평년 금액', '평년 금액 증감률(%)']
    
    for col in numeric_cols:
        if col in work_df.columns:
            work_df[col] = pd.to_numeric(work_df[col], errors='coerce')
    
    # 4️⃣ 일자별로 품목들을 통합 (가중평균 사용)
    # 반입량을 가중치로 사용하여 더 정확한 통합값 계산
    
    integrated_data = []
    
    for date in sorted(work_df['거래일자'].unique()):
        daily_data = work_df[work_df['거래일자'] == date]
        
        # 유효한 반입량이 있는 데이터만 사용
        valid_data = daily_data[daily_data['반입량'].notna() & (daily_data['반입량'] > 0)]
        
        if len(valid_data) == 0:
            continue
            
        # 반입량을 가중치로 사용한 가중평균 계산
        total_quantity = valid_data['반입량'].sum()
        weights = valid_data['반입량'] / total_quantity
        
        # 각 컫럼별 가중평균 계산
        integrated_row = {'거래일자': date, '품목명': '깐마늘_통합'}
        
        # 반입량은 합계 사용
        integrated_row['반입량'] = total_quantity
        
        # 나머지 컬럼들은 가중평균 사용
        for col in numeric_cols:
            if col != '반입량' and col in valid_data.columns:
                valid_values = valid_data[valid_data[col].notna()]
                if len(valid_values) > 0:
                    # 해당 컬럼에 유효한 값이 있는 행들의 가중치 재계산
                    col_weights = valid_values['반입량'] / valid_values['반입량'].sum()
                    integrated_row[col] = (valid_values[col] * col_weights).sum()
                else:
                    integrated_row[col] = np.nan
        
        integrated_data.append(integrated_row)
    
    # 5️⃣ 결과 DataFrame 생성
    result_df = pd.DataFrame(integrated_data)
    
    if len(result_df) == 0:
        print("⚠️ 통합할 유효한 데이터가 없습니다.")
        return pd.DataFrame()
    
    # 날짜순 정렬
    result_df = result_df.sort_values('거래일자').reset_index(drop=True)
    
    # 6️⃣ 만약 전년/평년 데이터가 없다면 별도 계산
    if '전년 반입량' not in result_df.columns or result_df['전년 반입량'].isna().all():
        result_df = calculate_comparison_data(result_df)
    
    print(f"✅ 통합 완료: {len(result_df)}개 행")
    print_data_quality_summary(result_df)
    
    return result_df

def calculate_comparison_data(df):
    """
    전년대비, 평년대비 데이터를 계산하는 함수
    """
    print("\n=== 전년/평년 비교 데이터 계산 ===")
    
    df = df.copy()
    df['연도'] = df['거래일자'].dt.year
    df['월'] = df['거래일자'].dt.month
    df['일'] = df['거래일자'].dt.day
    
    # 7️⃣ 전년 동일 날짜 데이터 매칭
    df_prev = df.copy()
    df_prev['연도'] += 1  # 1년 뒤로 이동
    df_prev = df_prev.rename(columns={
        '반입량': '전년 반입량',
        '금액': '전년 금액'
    })[['연도', '월', '일', '전년 반입량', '전년 금액']]
    
    # 전년 데이터 병합
    df = pd.merge(df, df_prev, on=['연도', '월', '일'], how='left')
    
    # 8️⃣ 평년 데이터 계산 (월별 평균)
    monthly_avg = df.groupby('월')[['반입량', '금액']].mean().reset_index()
    monthly_avg = monthly_avg.rename(columns={
        '반입량': '평년 반입량',
        '금액': '평년 금액'
    })
    
    # 평년 데이터 병합
    df = pd.merge(df, monthly_avg, on='월', how='left')
    
    # 9️⃣ 증감률 계산
    def safe_percentage(current, previous):
        """안전한 증감률 계산"""
        mask = (pd.notna(current)) & (pd.notna(previous)) & (previous != 0)
        result = pd.Series(np.nan, index=current.index)
        result[mask] = ((current[mask] - previous[mask]) / previous[mask]) * 100
        return result
    
    df['전년 반입량 증감률(%)'] = safe_percentage(df['반입량'], df['전년 반입량'])
    df['전년 금액 증감률(%)'] = safe_percentage(df['금액'], df['전년 금액'])
    df['평년 반입량 증감률(%)'] = safe_percentage(df['반입량'], df['평년 반입량'])
    df['평년 금액 증감률(%)'] = safe_percentage(df['금액'], df['평년 금액'])
    
    # 불필요한 컬럼 제거
    df = df.drop(['연도', '월', '일'], axis=1)
    
    return df

def print_data_quality_summary(df):
    """데이터 품질 요약 출력"""
    print("\n📊 데이터 품질 요약:")
    print(f"총 데이터 수: {len(df)}개")
    print(f"날짜 범위: {df['거래일자'].min().strftime('%Y-%m-%d')} ~ {df['거래일자'].max().strftime('%Y-%m-%d')}")
    
    # 각 컬럼별 유효 데이터 비율
    for col in df.columns:
        if col not in ['거래일자', '품목명']:
            valid_count = df[col].notna().sum()
            valid_rate = valid_count / len(df) * 100
            print(f"  {col}: {valid_count}개 ({valid_rate:.1f}%)")

def validate_extended_data(df):
    """확장된 데이터 검증"""
    print("\n=== 확장 데이터 검증 ===")
    
    if len(df) == 0:
        print("❌ 검증할 데이터가 없습니다.")
        return
    
    # 최근 데이터 미리보기
    print("\n📋 최근 10개 데이터:")
    recent_data = df.tail(10)
    
    # 주요 컬럼만 출력
    display_cols = ['거래일자', '반입량', '전년 반입량', '전년 반입량 증감률(%)', 
                   '금액', '전년 금액', '전년 금액 증감률(%)']
    available_display_cols = [col for col in display_cols if col in df.columns]
    
    print(recent_data[available_display_cols].to_string(index=False))
    
    # 통계 요약
    valid_data = df.dropna(subset=['반입량', '금액'])
    if len(valid_data) > 0:
        print(f"\n📈 유효 데이터 통계 ({len(valid_data)}개):")
        print("반입량:")
        print(f"  평균: {valid_data['반입량'].mean():.1f}kg")
        print(f"  범위: {valid_data['반입량'].min():.1f} ~ {valid_data['반입량'].max():.1f}kg")
        
        print("금액:")
        print(f"  평균: {valid_data['금액'].mean():,.0f}원")
        print(f"  범위: {valid_data['금액'].min():,.0f} ~ {valid_data['금액'].max():,.0f}원")
        
        # 증감률 통계 (유효한 값만)
        for rate_col in ['전년 반입량 증감률(%)', '전년 금액 증감률(%)', 
                        '평년 반입량 증감률(%)', '평년 금액 증감률(%)']:
            if rate_col in df.columns:
                valid_rates = df[rate_col].dropna()
                if len(valid_rates) > 0:
                    print(f"{rate_col}: 평균 {valid_rates.mean():.1f}% (범위: {valid_rates.min():.1f}% ~ {valid_rates.max():.1f}%)")

# 🚀 실행 코드
print("=== 확장된 깐마늘 데이터 통합 시작 ===")

# 1. 데이터 로드
df = pd.read_csv('깐마늘머지.csv', encoding='utf-8')
print(f"원본 데이터: {df.shape}")

# 2. 확장된 통합 실행
integrated_garlic_extended = integrate_garlic_data_extended(df)

# 3. 데이터 검증
validate_extended_data(integrated_garlic_extended)

# 4. CSV 저장
if len(integrated_garlic_extended) > 0:
    integrated_garlic_extended.to_csv('깐마늘_통합_확장_데이터.csv', index=False, encoding='utf-8-sig')
    print(f"\n💾 파일 저장 완료: '깐마늘_통합_확장_데이터.csv'")
else:
    print("⚠️ 저장할 데이터가 없습니다.")

print("\n🎉 모든 처리가 완료되었습니다!")


=== 확장된 깐마늘 데이터 통합 시작 ===
원본 데이터: (10962, 12)
=== 확장된 깐마늘 데이터 통합 시작 ===
대상 품목 데이터: 10962개 행
사용 가능한 컬럼: ['거래일자', '품목명', '반입량', '전년 반입량', '전년 반입량 증감률(%)', '금액', '전년 금액', '전년 금액 증감률(%)', '평년 반입량', '평년 반입량 증감률(%)', '평년 금액', '평년 금액 증감률(%)']
✅ 통합 완료: 3051개 행

📊 데이터 품질 요약:
총 데이터 수: 3051개
날짜 범위: 2015-06-30 ~ 2025-06-30
  반입량: 3051개 (100.0%)
  전년 반입량: 2489개 (81.6%)
  전년 반입량 증감률(%): 3051개 (100.0%)
  금액: 3051개 (100.0%)
  전년 금액: 2490개 (81.6%)
  전년 금액 증감률(%): 3051개 (100.0%)
  평년 반입량: 2596개 (85.1%)
  평년 반입량 증감률(%): 3051개 (100.0%)
  평년 금액: 2596개 (85.1%)
  평년 금액 증감률(%): 3051개 (100.0%)

=== 확장 데이터 검증 ===

📋 최근 10개 데이터:
      거래일자     반입량       전년 반입량  전년 반입량 증감률(%)           금액        전년 금액  전년 금액 증감률(%)
2025-06-19  7940.0  7400.000000       7.300000 6.498750e+07 3.843100e+07     69.100000
2025-06-20  9100.0  5860.000000      55.300000 7.156500e+07 3.250500e+07    120.200000
2025-06-21  8880.0  5760.000000      54.200000 6.698250e+07 3.110000e+07    115.400000
2025-06-23 10700.0          NaN       0.00

In [30]:
# 이미지 순서대로 컬럼 재배치
integrated_garlic_extended = integrated_garlic_extended[[
    '거래일자', '품목명', '반입량', '금액', 
    '전년 반입량', '전년 반입량 증감률(%)', 
    '전년 금액', '전년 금액 증감률(%)',
    '평년 반입량', '평년 반입량 증감률(%)', 
    '평년 금액', '평년 금액 증감률(%)'
]]

print("✅ 컬럼 순서 변경 완료!")
print(integrated_garlic_extended.columns.tolist())

integrated_garlic_extended.to_csv('깐마늘_통합_최종_데이터.csv', index=False, encoding='utf-8-sig')

✅ 컬럼 순서 변경 완료!
['거래일자', '품목명', '반입량', '금액', '전년 반입량', '전년 반입량 증감률(%)', '전년 금액', '전년 금액 증감률(%)', '평년 반입량', '평년 반입량 증감률(%)', '평년 금액', '평년 금액 증감률(%)']
